# Lists and Binary Trees Evaluation report
- Academic Year: 2019/20
- Faculty of Computing - Artificial Intelligence - Semester 2
- Full name and student id: Kacper Trębacz 145453
- Subject: Algorithms and Data Structures


The main purpose of the excercise was to evaluate performance of add, search and delete operations on lists, binary trees and avl trees. I divided this exercise into 2 main sections and one initial section concerning data used in exercise. At first lets look data that are data that we are going to use.

In [2]:
from random import randint,sample
import sys
import matplotlib.pyplot as plt
import numpy as np
import time 
import math
import random
import string
import copy
import pandas as pd
import json
sys.setrecursionlimit(10**9)
plt.rcParams['figure.figsize'] = (10.0, 6.0)

First, lets see declarations of functions that were used to generate data:

In [ ]:
def prepare_student(idx):  
    return [''.join([random.choice(string.ascii_letters) for _ in range(12)]),''.join([random.choice(string.ascii_letters) for _ in range(12)]),idx]

def prepare_students(amount):
    return [prepare_student(idx) for idx in range(1000000,1000000+amount)]

def prepare_test_dataset(maximum,jump=1):
    X = [amount for amount in range(1,maximum,jump)]
    Y = [prepare_students(amount) for amount in X]
    return X,Y

def generate_and_write_to_file():
    with open('Ex1.json','w') as file:
        json.dump(prepare_test_dataset(40002,2000), file)
    with open('Ex2.json','w') as file:
        json.dump(prepare_test_dataset(10002,250), file)
    with open('Ex2-2.json','w') as file:
        json.dump(prepare_test_dataset(100002,2500),file)
generate_and_write_to_file()

Data data entry that we are going to store represents a student with firstname (12 characters), surname (12 characters) and index (7 digits). The values in structures are sorted by index. 

After generating those data it was stored in 3 files: 'Ex1.json', 'Ex2.json', 'Ex2-2.json'


# Section 1

In this exercise we are going to compare storage, search and deletion time for sorted Linked List and Binary Tree

## Structure Definitions

### Sorted Linked List

In [4]:
class Node:
    def __init__(self,surname,firstname,index):
        self.surname = surname
        self.firstname = firstname
        self.index = index
        self.next_node = None

        
class SortedList:
    def __init__(self):
        self.next_node = None
    def add(self,element):
        new_node = Node(element[0],element[1],element[2])
        if not self.next_node:
            self.next_node = new_node
            return
        node = self
        while(node.next_node and node.next_node.index < new_node.index):
            node = node.next_node
        temp_node = node.next_node
        node.next_node = new_node
        new_node.next_node = temp_node
    
    def get(self,index):
        node = self.next_node
        while(node and node.index < index):
            node = node.next_node
        if(node and node.index == index):
            return node
        else:
            return None
    def delete(self,index):
        node = self
        while(node.next_node and node.next_node.index < index):
            node = node.next_node
        if(node.next_node and node.next_node.index == index):
            temp = node.next_node
            node.next_node = node.next_node.next_node#
            del(temp)
        else:
            raise BaseException("Element not found")
    def print_all(self):
        node = self.next_node
        while(node):
            print(node.surname+" "+node.firstname+" "+str(node.index))
            node = node.next_node
        

### Binary Tree

In [37]:
class TreeNode:
    def __init__(self,surname, firstname, index,left,right):
        self.left=left
        self.right=right
        self.surname = surname
        self.firstname = firstname
        self.index = index
    def print_me(self,level):
        print(''.join(["-" for i in range(level)])+str(self.index)+" " + self.firstname + " "+ self.surname)
        if self.left:
            print(''.join(["-" for i in range(level)])+'left')
            self.left.print_me(level+1)
        if self.right:
            print(''.join(["-" for i in range(level)])+'right')
            self.right.print_me(level+1)
    def count(self):
        suma = 0
        if self.left:
            suma+=self.left.count()
        if self.right:
            suma+=self.right.count()
        return suma+1
    def delete(self,index):
        if self.index == index:
            if self.left and self.right:
                next_node = self.find_smallest(self.right)
                self.index = next_node.index
                self.surname = next_node.surname
                self.firstname = next_node.surname
            elif self.left:
                return self.left
            elif self.right:
                return self.right
            else:
                return None
        elif index < self.index:
            if not self.left:
                raise BaseException("Index not found")
            self.left = self.left.delete(index)
        else:
            if not self.right:
                raise BaseException("Index not found")
            self.right = self.right.delete(index)
        return self
    def find_smallest(self,node):
        parent =self
        child = node
        if not node.left:
            parent.right = node.right
            return node
        while node.left:
            parent = node
            node =node.left
        parent.left = node.right
        return node
    
    def show(self):
        print(str(self.index) + " " + self.firstname + " "+ self.surname)
        
    def pre_order(self):
        self.show()
        if self.left:
            self.left.pre_order()
        if self.right:
            self.right.pre_order()
        
    def post_order(self):
        if self.left:
            self.left.post_order()
        if self.right:
            self.right.post_order()
        self.show()
        
    def in_order(self):
        if self.left:
            self.left.in_order()
        self.show()
        if self.right:
            self.right.in_order()

class Tree:
    def __init__(self):
        self.root = None
    def add(self,element):
        new_node = TreeNode(element[1],element[2],element[0],None,None);
        if not self.root:
            self.root = new_node
            return
        cur_node = self.root
        while True:
            if new_node.index < cur_node.index:
                if cur_node.left:
                    cur_node=cur_node.left
                else:
                    cur_node.left = new_node
                    break
            else:
                if cur_node.right:
                    cur_node = cur_node.right                
                else:
                    cur_node.right = new_node
                    break
    def get(self,index):
        cur_node = self.root
        while cur_node:
            if index < cur_node.index:
                cur_node = cur_node.left
            elif index > cur_node.index:
                cur_node = cur_node.right
            else:
                return cur_node
        raise BaseException("No node with this id")
    def delete(self,index):
        self.root=self.root.delete(index)


    def get_min_node(self,node):
        parent = node
        while parent.left and parent.left.left:
            parent = parent.left
        if parent.left:
            temp = parent.left
            parent.left = None
            return temp
        return parent
        
    def print_all(self):
        if self.root:
            self.root.print_me(0)
        else:
            print("Empty tree")
    def count(self):
        if not self.root:
            return 0
        return self.root.count()
    def pre_order(self):
        self.root.pre_order()
    
    def post_order(self):
        self.root.post_order()
        
    def in_order(self):
        self.root.in_order()
        

In [38]:
text ="00004 Asssskskydws Rwnsjedinsar\n00007 Dejdkantskre Kwbsrjdogsaw\n00001 Pnhatdeoicnq Swifnesarksw\n00005 Iaserjddswut Cwjdernswero\n00002 Aqwevytskdyt Swdbwiednawe\n00009 Ldarwncdrisa Swropdcufsaz\n00003 Usfrwjdloers Swertpflsawr\n00010 Jhserpdawerl Jwrdlaserods\n00006 Mnstarwjcdri Baaserdokews\n00008 Msderlfdswie Qzserfsiogaw\n00011 Haserodxcswt Msraskermsqa"
data = list(map(lambda x:x.split(" "), text.split("\n")))
print(data)

[['00004', 'Asssskskydws', 'Rwnsjedinsar'], ['00007', 'Dejdkantskre', 'Kwbsrjdogsaw'], ['00001', 'Pnhatdeoicnq', 'Swifnesarksw'], ['00005', 'Iaserjddswut', 'Cwjdernswero'], ['00002', 'Aqwevytskdyt', 'Swdbwiednawe'], ['00009', 'Ldarwncdrisa', 'Swropdcufsaz'], ['00003', 'Usfrwjdloers', 'Swertpflsawr'], ['00010', 'Jhserpdawerl', 'Jwrdlaserods'], ['00006', 'Mnstarwjcdri', 'Baaserdokews'], ['00008', 'Msderlfdswie', 'Qzserfsiogaw'], ['00011', 'Haserodxcswt', 'Msraskermsqa']]


In [39]:
tr = Tree()
for el in data:
    tr.add(el)
    
print("In order")
tr.in_order()
print("Pre order")
tr.pre_order()
print("Post order")
tr.post_order()
print("me")
tr.print_all()

In order
00001 Swifnesarksw Pnhatdeoicnq
00002 Swdbwiednawe Aqwevytskdyt
00003 Swertpflsawr Usfrwjdloers
00004 Rwnsjedinsar Asssskskydws
00005 Cwjdernswero Iaserjddswut
00006 Baaserdokews Mnstarwjcdri
00007 Kwbsrjdogsaw Dejdkantskre
00008 Qzserfsiogaw Msderlfdswie
00009 Swropdcufsaz Ldarwncdrisa
00010 Jwrdlaserods Jhserpdawerl
00011 Msraskermsqa Haserodxcswt
Pre order
00004 Rwnsjedinsar Asssskskydws
00001 Swifnesarksw Pnhatdeoicnq
00002 Swdbwiednawe Aqwevytskdyt
00003 Swertpflsawr Usfrwjdloers
00007 Kwbsrjdogsaw Dejdkantskre
00005 Cwjdernswero Iaserjddswut
00006 Baaserdokews Mnstarwjcdri
00009 Swropdcufsaz Ldarwncdrisa
00008 Qzserfsiogaw Msderlfdswie
00010 Jwrdlaserods Jhserpdawerl
00011 Msraskermsqa Haserodxcswt
Post order
00003 Swertpflsawr Usfrwjdloers
00002 Swdbwiednawe Aqwevytskdyt
00001 Swifnesarksw Pnhatdeoicnq
00006 Baaserdokews Mnstarwjcdri
00005 Cwjdernswero Iaserjddswut
00008 Qzserfsiogaw Msderlfdswie
00011 Msraskermsqa Haserodxcswt
00010 Jwrdlaserods Jhserpdawerl
00009 Swro

## Time Measurement

For every structure and every n we add n elements to the structure and measure time of this operation and divide by n. We do the same for search and delete operation.

When we shuffle elements in array we used seed based on number of elements so every structure for every n gets the same sequence of elements

In [6]:
def measure_time(students,store):
    s_copy = students.copy()
    random.seed(len(s_copy))
    random.shuffle(s_copy)
    start = time.time()
    for s in s_copy:
        store.add(s)
    add_time = (time.time() - start)/len(s_copy)
    random.seed(len(s_copy)*2)
    random.shuffle(s_copy)
    start = time.time()
    for s in s_copy:
        store.get(s[2])
    get_time = (time.time() - start)/len(s_copy)
    random.seed(len(s_copy)*3)
    random.shuffle(s_copy)
    start = time.time()
    for s in s_copy:
        store.delete(s[2])
    delete_time = (time.time() - start)/len(s_copy)
    return add_time,get_time,delete_time

## Evaluation function

In [7]:
def perform_test_single_store(store,X,Y):
    adds,gets,deletes=[],[],[]
    for y in Y:
        add,get,delete = measure_time(y,store)
        adds.append(add)
        gets.append(get)
        deletes.append(delete)
    return adds,gets,deletes

def plot_plot(X,Ys,labels,title):
    for i in range(len(Ys)):
        plt.plot(X,Ys[i],label=labels[i])
    plt.legend()
    plt.xlabel('Number of Elements')
    plt.ylabel('Time (s)')
    plt.title(title)
    plt.show()
    
    
def perform_test(stores,X,Y,labels):
    adds,gets,deletes =[],[],[]
    for store in stores:
        add,get,delete = perform_test_single_store(store,X,Y)
        adds.append(add)
        gets.append(get)
        deletes.append(delete)
    plot_plot(X,adds,labels,"Addition time")
    plot_plot(X,gets,labels,"Finding time")
    plot_plot(X,deletes,labels,"Deletion time")
    data = {'N':X}
    for i in range(len(stores)):
        data[labels[i] + ' Average addition time (s)'] = adds[i]
        data[labels[i] + ' Average search time (s)'] = gets[i]
        data[labels[i] + ' Deletion time (s)'] = deletes[i]
    df = pd.DataFrame(data)
    df.set_index('N',inplace=True)
    display(df)

Now we can run the experiment. We read data from 'Ex1.json' where we have 21 collections of students of size from 1 to 400001 and we insert, search and delete them from Lisked List and Binary Tree. Results are presented in 3 separate graphs for each operation and in 1 table with all data.

In [ ]:
with open('Ex1.json', 'r') as file:
    X,Y = json.load(file)
perform_test([SortedList(), Tree()],X,Y,["List","Tree"])

# Section 2

In this section we are going to compare storage and search time for sorted Linked List, Binary Tree, and Balanced Binary Tree(AVL)

I am aware that we were supposed to just measure search time but I realised it after I have also implemented measuring addition time, so I thought that I would add another chart and make some extra conclusions.

### Balance binary tree (AVL) definition

In [ ]:
class AVLNode:
    def __init__(self,surname, firstname, index,left,right):
        self.left=left
        self.right=right
        self.surname = surname
        self.firstname = firstname
        self.index = index
        self.height = 1
    def print_me(self,level):
        print('-'.join(["-" for i in range(level)])+self.surname+self.firstname+str(self.index))
        if self.left:
            print('-'.join(["-" for i in range(level)])+'left')
            self.left.print_me(level+1)
        if self.right:
            print('-'.join(["-" for i in range(level)])+'right')
            self.right.print_me(level+1)

class AVLTree:
    def __init__(self):
        self.root = None
    def add(self,element):
        new_node = AVLNode(element[0],element[1],element[2],None,None);
        self.root = self.add_recursive(self.root,new_node)
        
    def add_recursive(self,root,new_node):
        if not root:
            return new_node
        if new_node.index < root.index:
            root.left = self.add_recursive(root.left,new_node)
        else:
            root.right = self.add_recursive(root.right,new_node)
        
        root.height = 1 + max(self.calculate_height(root.left),self.calculate_height(root.right))
        balance = self.calculate_balance(root)
        
        if balance > 1 and new_node.index  < root.left.index:
            return self.rotate_right(root)
        if balance < -1 and new_node.index > root.right.index:
            return self.rotate_left(root)
        if balance > 1 and new_node.index > root.left.index:
            root.left = self.rotate_left(root.left)
            return self.rotate_right(root)
        if balance < -1 and new_node.index < root.right.index:
            root.right = self.rotate_right(root.right)
            return self.rotate_left(root)
        return root
        
    def rotate_left(self,node):
        node2 = node.right
        temp = node2.left
        node2.left = node
        node.right = temp
        
        node.height = 1 + max(self.calculate_height(node.left),self.calculate_height(node.right))
        node2.height = 1 + max(self.calculate_height(node2.left),self.calculate_height(node2.right))
        return node2
    
    def rotate_right(self,node):
        node2 = node.left
        temp = node2.right
        node2.right = node
        node.left = temp
        
        node.height = 1 + max(self.calculate_height(node.left),self.calculate_height(node.right))
        node2.height = 1 + max(self.calculate_height(node2.left),self.calculate_height(node2.right))
        return node2
        
        
    def calculate_balance(self,node):
        if not node:
            return 0
        return self.calculate_height(node.left)-self.calculate_height(node.right)
    def calculate_height(self,node):
        if not node:
            return 0
        
        return node.height
        
    def get(self,index):
        cur_node = self.root
        while cur_node:
            if index < cur_node.index:
                cur_node = cur_node.left
            elif index > cur_node.index:
                cur_node = cur_node.right
            else:
                return cur_node
        raise BaseException("No node with this id")
    def print_all(self):
        if self.root:
            self.root.print_me(0)
        else:
            print("Empty tree")
    def count(self):
        if not self.root:
            return 0
        return self.root.count()
    

## Time measurement

For every structure and every n we add n elements to the structure and measure time of this operation and divide by n. We do the same for search operation.

When we shuffle elements in array we use seed based on the number of elements so every structure for every n gets the same sequence of elements

In [ ]:
def measure_time_2(students,store_class):
    s_copy = students.copy()
    random.seed(len(s_copy))
    random.shuffle(s_copy)
    store=store_class()
    start = time.time()
    for s in s_copy:
        store.add(s)
    add_time = (time.time() - start)/len(students)
    random.seed(len(s_copy)*2)
    random.shuffle(s_copy)
    start = time.time()
    for s in s_copy:
        store.get(s[2])
    get_time = (time.time() - start)/len(s_copy)
    return add_time,get_time


## Evaluation function

In [ ]:
def perform_test_single_store_2(store,X,Y):
    adds,gets=[],[]
    for y in Y:
        add,get = measure_time_2(y,store)
        adds.append(add)
        gets.append(get)
    return adds,gets

def perform_test_2(stores,X,Y,labels):
    adds,gets =[],[]
    for store in stores:
        add,get = perform_test_single_store_2(store,X,Y)
        adds.append(add)
        gets.append(get)
    plot_plot(X,adds,labels,"Addition time")
    plot_plot(X,gets,labels,"Finding time")
    data = {'N':X}
    for i in range(len(stores)):
        data[labels[i] + ' Average addition time (s)'] = adds[i]
        data[labels[i] + ' Average search time (s)'] = gets[i]
    df = pd.DataFrame(data)
    df.set_index('N',inplace=True)
    display(df)
    
    

Now we can run the experiment. We read data from 'Ex2.json' where we have 41 collections of students of size from 1 to 100001 and we insert and search for them from Lisked List, Binary Tree and Balanced Binary Tree(AVL). Results are presented in 2 separate graphs for each operation and in 1 table with all data.

In [ ]:
with open('Ex2.json', 'r') as file:
    X,Y = json.load(file)
perform_test_2([SortedList, Tree, AVLTree],X,Y,["List","Tree", "AVLTree"])

Now we do the same, but just for Binary Tree and Balance Binary Tree with 41 collections of students of size from 1 to 100001

In [ ]:
with open('Ex2-2.json', 'r') as file:
    X,Y = json.load(file)
perform_test_2([Tree, AVLTree],X,Y,["Tree", "AVLTree"])

# Conclusions

In this setting we can clearly see that Linked Lists are no good for storing values in a sorted order. Insertion, searching and deletion time are huge, when compared to Binary search trees. It means that they should not be used in such situations. Their key advantage is that they are easy to implement, but considering the fact that all structures implemented in this exercise are also implemented in standard libraries of every programming language this advantage becomes meamingless for similar tasks. 

When it comes to comparing Binary Search Tree (BST) and Balanced Binary Search Tree (here AVL), we can see 2 key differences. BST has much better addition time and AVL has slightly better search time. It is due to the fact that AVL needs to perform additional operations in order to remain balanced and allow for ideal O(logn) search time. Lack of those operations in BST leeds to the situation when tree is not divided equally so the base of logarithm becomes less than 2 and search complexity increases. 

Previous observations show that we should carefully consider when choosing between AVL and BST. If we know that there are search operations are going to occur much more frequently, then we should choose AVL. However, if there are only a few times more search operations or and we know that values will not be sorted we should use BST